In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from google.colab import drive
from tensorflow import keras
from tensorflow.keras import layers
import pickle
from keras import backend as K

In [ ]:
# Load the dataset
drive.mount('/content/drive')
df =pd.read_csv("/content/drive/MyDrive/SWM/restructured_data.csv")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Split the dataset into input and output
X = df['Data']
#X.append(data['reformulated_tweets'])
Y=df['Stance']
#Y.append(data['stance'])
Y = pd.get_dummies(Y).values
#targets = df['target'].unique()

In [ ]:
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(X)
X = tokenizer.texts_to_sequences(X)

# Pad the sequences
maxlen = 100
X = pad_sequences(X, padding='post', maxlen=maxlen)

# Split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)



In [ ]:
# Input for variable-length sequences of integers
inputs = keras.Input(shape=(None,), dtype="int32")
# Embed each integer in a 128-dimensional vector
x = layers.Embedding(10000, 128)(inputs)
# Add 2 bidirectional LSTMs
x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
# Add a classifier
outputs = layers.Dense(3, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 128)         1280000   
                                                                 
 bidirectional (Bidirectiona  (None, None, 128)        98816     
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 128)              98816     
 nal)                                                            
                                                                 
 dense (Dense)               (None, 3)                 387       
                                                                 
Total params: 1,478,019
Trainable params: 1,478,019
Non-train

In [ ]:

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
model.compile("adam", "categorical_crossentropy", metrics=["accuracy",f1_m,precision_m, recall_m])

In [ ]:
# # Define the model
# model = Sequential()
# model.add(Embedding(10000, 128, input_length=maxlen))
# model.add(Bidirectional(LSTM(64)))
# model.add(Dense(3, activation='softmax'))

# # Compile the model
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
model.fit(X_train, Y_train, batch_size=64, epochs=25, validation_data=(X_test, Y_test))

Epoch 1/25
36/36 [==============================] - 24s 297ms/step - loss: 1.0116 - accuracy: 0.5220 - f1_m: 0.5419 - precision_m: 0.5142 - recall_m: 0.5758 - val_loss: 0.9818 - val_accuracy: 0.5009 - val_f1_m: 0.5261 - val_precision_m: 0.4500 - val_recall_m: 0.6335
Epoch 2/25
36/36 [==============================] - 6s 162ms/step - loss: 0.7942 - accuracy: 0.6273 - f1_m: 0.6515 - precision_m: 0.5795 - recall_m: 0.7469 - val_loss: 0.9435 - val_accuracy: 0.5648 - val_f1_m: 0.5813 - val_precision_m: 0.5205 - val_recall_m: 0.6592
Epoch 3/25
36/36 [==============================] - 6s 172ms/step - loss: 0.5665 - accuracy: 0.7059 - f1_m: 0.7455 - precision_m: 0.6240 - recall_m: 0.9267 - val_loss: 1.0276 - val_accuracy: 0.4849 - val_f1_m: 0.5444 - val_precision_m: 0.4534 - val_recall_m: 0.6813
Epoch 4/25
36/36 [==============================] - 5s 123ms/step - loss: 0.4610 - accuracy: 0.7503 - f1_m: 0.7698 - precision_m: 0.6383 - recall_m: 0.9701 - val_loss: 1.2310 - val_accuracy: 0.5417 - v

In [ ]:
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
score

Test loss: 2.3326191902160645
Test accuracy: 0.577264666557312


[2.3326191902160645,
 0.577264666557312,
 0.5948789715766907,
 0.5416694283485413,
 0.6606359481811523]

In [ ]:
print("AGAINST:",Y[0])
print("FAVOR",Y[10])
print("NONE",Y[9])

AGAINST: [1 0 0]
FAVOR [1 0 0]
NONE [1 0 0]


In [ ]:
# Predict the stance of new texts
new_texts = ["Women are smart", "Feminism is a myth"]
new_targets = ["Feminist Movement","Feminist Movement","Feminist Movement"]
new_texts = tokenizer.texts_to_sequences(new_texts)
new_texts = pad_sequences(new_texts, padding='post', maxlen=maxlen)
predictions = []
for i in range(len(new_texts)):
  pred = model.predict(np.array([new_texts[i]]))
  print(pred)


1/1 [==============================] - 1s 1s/step
[[0.9991411  0.0513379  0.09877845]]
1/1 [==============================] - 0s 22ms/step
[[0.9992434  0.0485199  0.09780858]]


In [ ]:
from sklearn.metrics import classification_report

y_pred = model.predict(X_test, batch_size=64, verbose=1)
y_pred_bool = np.argmax(Y_test, axis=1)

print(classification_report(Y_test, y_pred_bool))

9/9 [==============================] - 0s 13ms/step


ValueError: ignored

In [ ]:
#pickle.dump(model, open("Bi-LSTM.pkl", 'wb'))
